In [2]:
import os

os.chdir("../")
%pwd

'c:\\Users\\anfe1\\OneDrive\\Escritorio\\carpetaANFE\\AI_ML_Projects\\Text-Summarization'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_chkpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [4]:
from textSummarization.constants import *
from textSummarization.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self) -> None:
        config_file_path = CONFIG_FILE_PATH
        params_file_path = PARAMS_FILE_PATH

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.training_arguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_chkpt= config.model_chkpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            per_device_eval_batch_size= params.per_device_eval_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps
        )

        return model_trainer_config

In [7]:
import torch

from transformers import (Trainer, TrainingArguments,
                          DataCollatorForSeq2Seq,
                          AutoModelForSeq2SeqLM, AutoTokenizer)
from datasets import load_from_disk

from textSummarization.logging import logger

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig) -> None:
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        logger.info(f"Device using: {device}")
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_chkpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_chkpt).to(device)
        data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

        #Loading Data
        dataset = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir= self.config.root_dir,
            num_train_epochs= self.config.num_train_epochs,
            warmup_steps= self.config.warmup_steps,
            per_device_train_batch_size= self.config.per_device_train_batch_size,
            per_device_eval_batch_size= self.config.per_device_eval_batch_size,
            weight_decay= self.config.weight_decay,
            logging_steps= self.config.logging_steps,
            evaluation_strategy= self.config.evaluation_strategy,
            eval_steps= self.config.eval_steps,
            save_steps= self.config.save_steps,
            gradient_accumulation_steps= self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model= model,
            args= trainer_args,
            tokenizer= tokenizer,
            data_collator= data_collator,
            train_dataset= dataset["test"],
            eval_dataset= dataset["validation"]
        )
        
        trainer.train()

        #Save Model
        # trainer.save_model(save_directory=os.path.join(self.config.root_dir,"pegasus_samsum_model"))
        model.save_pretrained(save_directory=os.path.join(self.config.root_dir,"pegasus_samsum_model"))

        #Save Tokenizer
        tokenizer.save_pretrained(save_directory=os.path.join(self.config.root_dir,"tokenizer"))


In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e